# 04-extract_tdms

This notebook extracts time delayed melody surface from the predominant melody of each audio recording. The feature was introduced in:

> Gulati S, Serrà J, Ganguli K K, Şentürk S, Serra X. Time-delayed melody surfaces for Rāga recognition. In Proceedings of the 17th International Society for Music Information Retrieval Conference (ISMIR 2016); 2016 Aug 7-11; New York City (NY). p. 751-7.


In [1]:
import mre
import logging

%load_ext autoreload
%autoreload 2


In [2]:
logger = logging.getLogger()
logger.info("Init logger")

INFO:root:Init logger


## Display TDMS features computed in the past

Unlike the other features, we experiment with different TDMS settings to optimize the feature for OTMM. We will create multiple runs with the same name.

We display the runs below:


In [7]:
run = mre.mlflow_common.get_runs_with_same_name(
    mre.data.TimeDelayedMelodySurface.EXPERIMENT_NAME,
    mre.data.TimeDelayedMelodySurface.RUN_NAME)
display(run)

,run_id,experiment_id,status,artifact_uri,start_time,end_time,tags.kernel_width,tags.mlflow.source.type,tags.mlflow.user,tags.step_size,tags.time_delay_index,tags.mlflow.source.name,tags.compression_exponent,tags.mlflow.runName
0,9c4eddc076fb416bb46c8cc5c137f644,1,FINISHED,/data/artifacts/1/9c4eddc076fb416bb46c8cc5c137...,2022-05-16 01:20:13.497000+00:00,2022-05-16 01:20:24.817000+00:00,None,LOCAL,sertan.senturk,25.0,1.0,/opt/conda/lib/python3.7/site-packages/ipykern...,None,time_delayed_melody_surface
1,f58abf754683402295164df3747e3ea1,1,FINISHED,/data/artifacts/1/f58abf754683402295164df3747e...,2022-05-16 01:17:14.190000+00:00,2022-05-16 01:17:23.611000+00:00,None,LOCAL,sertan.senturk,25.0,0.5,/opt/conda/lib/python3.7/site-packages/ipykern...,None,time_delayed_melody_surface
2,8e2e0b78f61941beb26d8f2b0a1e3019,1,FINISHED,/data/artifacts/1/8e2e0b78f61941beb26d8f2b0a1e...,2022-05-16 01:13:47.381000+00:00,2022-05-16 01:13:56.420000+00:00,None,LOCAL,sertan.senturk,25.0,0.25,/opt/conda/lib/python3.7/site-packages/ipykern...,None,time_delayed_melody_surface


In [15]:
run["tags.compression_exponent"].iloc[0]



'None'

# Read predominant melody filepaths from mlflow


In [4]:
melody_paths = mre.data.PredominantMelodyMakam.from_mlflow()
display(melody_paths[:5])

['/data/artifacts/1/2608c404325846309549b68e5020bd17/artifacts/006536f8-bf54-4cc0-a510-5a52456d09f8.npy',
 '/data/artifacts/1/2608c404325846309549b68e5020bd17/artifacts/009309d2-c260-4808-8f1d-44a5ddc6bc5f.npy',
 '/data/artifacts/1/2608c404325846309549b68e5020bd17/artifacts/00a48b5f-a35a-436c-a7a0-4438130f4abf.npy',
 '/data/artifacts/1/2608c404325846309549b68e5020bd17/artifacts/00ab81ec-07f8-47ba-9610-47ad56393eb9.npy',
 '/data/artifacts/1/2608c404325846309549b68e5020bd17/artifacts/00be9a4b-b85f-4601-a6d5-9ce1f5b3f91c.npy']

# Read tonic annotations from mlflow

In [5]:
annotations = mre.data.Annotation.from_mlflow()
tonic_frequencies = annotations.data.set_index("mbid")["tonic"]
tonic_frequencies.head()


mbid
00f1c6d9-c8ee-45e3-a06f-0882ebcb4e2f    256.0
168f7c75-84fb-4316-99d7-acabadd3b2e6    115.2
24f549dd-3fa4-4e9b-a356-778fbbfd5cad    232.5
407bb0b4-f19b-42ab-8c0a-9f1263126951    233.5
443819eb-6092-420c-bd86-d946a0ad6555    219.6
Name: tonic, dtype: float64

# Extract TDMSs from normalized predominant melody features and log to mlflow


In [6]:
for tau in [0.25, 0.5, 1.0]:
    tdms_params = {
        "time_delay_index": tau,
        # no compression; will be done later during the experiments
        "compression_exponent": None,
        # smoothing; will be done later during the experiments
        "kernel_width": None,
    }
    logger.info(f"Computing TDMS with parameters: {tdms_params}")
    tdms = mre.data.TimeDelayedMelodySurface(
        **tdms_params
    )
    tdms.transform(melody_paths, tonic_frequencies)
    tdms.log()


INFO:root:Computing TDMS with parameters: {'time_delay_index': 0.25, 'compression_exponent': None, 'kernel_width': None}
100%|██████████| 1000/1000 [03:12<00:00,  5.20it/s]
No runs with the name time_delayed_melody_surface in experiment data_processing
INFO:root:Computing TDMS with parameters: {'time_delay_index': 0.5, 'compression_exponent': None, 'kernel_width': None}
100%|██████████| 1000/1000 [03:17<00:00,  5.06it/s]
INFO:root:Computing TDMS with parameters: {'time_delay_index': 1.0, 'compression_exponent': None, 'kernel_width': None}
100%|██████████| 1000/1000 [02:49<00:00,  5.89it/s]
